In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFDS and Rock, Paper, Scissors

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%203%20-%20TensorFlow%20Datasets/Week%201/Exercises/TFDS_Week1_Exercise.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/lmoroney/dlaicourse/blob/master/TensorFlow%20Deployment/Course%203%20-%20TensorFlow%20Datasets/Week%201/Exercises/TFDS_Week1_Exercise.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

In this week's exercise you will be working with TFDS and the rock-paper-scissors dataset. You'll do a few tasks such as exploring the info of the dataset in order to figure out the name of the splits. You'll also write code to see if the dataset supports the new S3 API before creating your own versions of the dataset.

## Setup

In [1]:
try:
    %tensorflow_version 2.x
except:
    pass

In [2]:
# Use all imports
import tensorflow as tf
import tensorflow_datasets as tfds

print("\u2022 Using TensorFlow Version:", tf.__version__)

• Using TensorFlow Version: 2.4.0


## Extract the Rock, Paper, Scissors Dataset

In the cell below, you will extract the `rock_paper_scissors` dataset and then print its info. Take note of the splits, what they're called, and their size.

In [10]:
# EXERCISE: Use tfds.load to extract the rock_paper_scissors dataset.

filePath = "tmp"
data, info = tfds.load(name="rock_paper_scissors", with_info=True, data_dir=filePath, as_supervised=True)
print(info)

Shuffling and writing examples to tmp/rock_paper_scissors/3.0.0.incomplete5GGUDS/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to tmp/rock_paper_scissors/3.0.0.incomplete5GGUDS/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to tmp/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    version=3.0.0,
    description='Images of hands playing rock, paper, scissor game.',
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=2892,
    splits={
        'test': 372,
        'train': 2520,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",
    redistribution_info=,
)



In [7]:
# EXERCISE: In the space below, write code that iterates through the splits
# without hardcoding any keys. The code should extract 'test' and 'train' as
# the keys, and then print out the number of items in the dataset for each key. 
# HINT: num_examples property is very useful here.

for set in data:
  print(set,":",info.splits[set].num_examples)

# EXPECTED OUTPUT
# test:372
# train:2520

test : 372
train : 2520


## Use the New S3 API

Before using the new S3 API, you must first find out whether the `rock_paper_scissors` dataset implements the new S3 API. In the cell below you should use version `3.*.*` of the `rock_paper_scissors` dataset.

In [20]:
# EXERCISE: In the space below, use the tfds.builder to fetch the
# rock_paper_scissors dataset and check to see if it supports the
# new S3 API. 
# HINT: The builder should 'implement' something

rps_builder = tfds.builder("rock_paper_scissors:3.*.*", data_dir=filePath)

 # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*")

# print(rps_builder.version.implements(tfds.core.Experiment.S3))

# Expected output:
# True

## Create New Datasets with the S3 API

Sometimes datasets are too big for prototyping. In the cell below, you will create a smaller dataset, where instead of using all of the training data and all of the test data, you instead have a `small_train` and `small_test` each of which are comprised of the first 10% of the records in their respective datasets.

In [16]:
# EXERCISE: In the space below, create two small datasets, `small_train`
# and `small_test`, each of which are comprised of the first 10% of the
# records in their respective datasets.

small_train =  tfds.load("rock_paper_scissors:3.*.*", split='train[:10%]',data_dir=filePath) 
small_test = tfds.load("rock_paper_scissors:3.*.*", split='test[:10%]',data_dir=filePath) 

# No expected output yet, that's in the next cell

In [17]:
# EXERCISE: Print out the size (length) of the small versions of the datasets.

print(len(list(small_train)))
print(len(list(small_test)))

# Expected output
# 252
# 37

252
37


The original dataset doesn't have a validation set, just training and testing sets. In the cell below, you will use TFDS to create new datasets according to these rules:

* `new_train`: The new training set should be the first 90% of the original training set.


* `new_test`: The new test set should be the first 90% of the original test set.


* `validation`: The new validation set should be the last 10% of the original training set + the last 10% of the original test set.

In [18]:
# EXERCISE: In the space below, create 3 new datasets according to
# the rules indicated above.

new_train = tfds.load("rock_paper_scissors:3.*.*", split='train[:90%]',data_dir=filePath)  # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*", data_dir=filePath)
print(len(list(new_train)))

new_test = tfds.load("rock_paper_scissors:3.*.*", split='test[:90%]',data_dir=filePath) # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*", data_dir=filePath)
print(len(list(new_test)))

validation = tfds.load("rock_paper_scissors:3.*.*", split='train[90%:] + test[90%:]',data_dir=filePath) # YOUR CODE HERE (Include the following arguments in your code: "rock_paper_scissors:3.*.*", data_dir=filePath)
print(len(list(validation)))

# Expected output
# 2268
# 335
# 289

2268
335
289
